In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
chunk_size = 1000
collection_name = f"mycollection_{chunk_size}"
embedding_model = "text-embedding-3-large"

top_k = 4
batch_size = 100

query="高町なのはがユーノから譲り受けたものは何ですか？"

### 埋め込みモデルの準備

In [3]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### ドキュメントの読み込みとチャンキング

In [4]:
from langchain_community.document_loaders import PDFMinerLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader(
    "./data/",
    glob="**/*.pdf",  # PDFファイルを対象に
    show_progress=True,
    loader_cls=PDFMinerLoader  # PDFMinerLoaderを使用
)

In [5]:
# ドキュメントの分割
documents = []
for doc in loader.load():
    # すべての改行を削除（単語の途中でチャンキングされることを防ぐため）
    doc.page_content = doc.page_content.replace("\n", "")
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        separators=[
            "。",
            "、",
            " ",
            "」",
            "）",
            "｝",
            "〕",
            "》"
        ],
        keep_separator="end"
    ).split_documents([doc])
    documents.extend(chunks)

print(f"合計チャンク数: {len(documents)}")

100%|██████████| 19/19 [00:53<00:00,  2.80s/it]

合計チャンク数: 598


In [6]:
for i, doc in enumerate(documents[:5]):
    print(f"\n=== Document {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== Document 1 ===
Content: ハリー‧ポッター (架空の⼈物)出典: フリー百科事典『ウィキペディア（Wikipedia）』ハリー‧ポッターシリーズ > ハリー‧ポッターシリーズの登場⼈物⼀覧 > ハリー‧ポッター (架空の⼈物)ハリー‧ジェームズ‧ポッター（英: Harry JamesPotter）は、J‧K‧ローリングの⼩説『ハリー‧ポッター』シリーズおよび、その派⽣作品に登場する架空の⼈物であり、同シリーズの主⼈公。ホグワーツ魔法魔術学校グリフィンドール寮の男⼦⽣徒となる。孤児として⺟親の親類の伯⺟夫婦の家で不遇な暮らしをして育った。11歳を迎える年のある⽇突然、ホグワーツから⼊学許可証が届いたのをきっかけに、亡くなった両親が魔法使いであったこと、そして出⽣時に下された予⾔により、闇の魔法使いヴォルデモートを倒す宿命を⾃分が負っていると告げられる。マグル界では⼀介の少年に過ぎない⽣活を送っていたが、魔法界では本⼈が⼾惑うほど重要な⼈物として、あまねく⼈々から知られている。⼀⼈前の魔法使いになるべく、同級⽣のロン‧ウィーズリーやハーマイオニー‧グレンジャーらとともに、ホグワーツにて学⽣⽣活を送りつつ、宿敵のヴォルデモートなどの闇の魔法使いたちによる数々の陰謀に⽴ち向かう冒険の⽇々を通して、たくましく成⻑していく姿が物語で描かれている。⼈物名前‧外⾒魔 法 界 で は 「 ⽣ き 残 っ た 男 の ⼦   (The  boy  wholived) 」と呼ばれる。髪の⽑は黒い癖⽑で、瞳は明るい緑⾊。⼩顔で細⾯で、近視のため丸眼鏡を着⽤。同年代に⽐べ⼩柄で痩せているが、第6巻『謎のプリンス』では前巻と⽐べて⾝⻑がかなり伸びたとされている。
文字数: 716
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 2 ===
Content: 髪の⽑は黒い癖⽑で、瞳は明るい緑⾊。⼩顔で細⾯で、近視のため丸眼鏡を着⽤。同年代に⽐べ⼩柄で痩せているが、第6巻『謎のプリンス』では前巻と⽐べて⾝⻑がかなり伸びたとされている。額にはヴォルデモートの強⼒な呪いによってつけハリー‧ポッターHarry Potter『ハリー‧ポッター』シリーズのキャラクターハリー‧ポッターを演じるダニエル‧ラドクリ

### VectorStoreの準備

In [7]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    # 既存のChromaコレクションを削除
    try:
        persistent_client.delete_collection(collection_name)
        print(f"既存のコレクション '{collection_name}' を削除しました")
    except ValueError as e:
        print(f"既存のコレクション '{collection_name}' は存在しませんでした")
    except Exception as e:
        print(f"コレクションの削除中にエラーが発生しました: {str(e)}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip"} 
    )
    print(f"コレクション '{collection_name}' を作成しました")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    # ドキュメントをバッチで追加
    print(f"コレクション '{collection_name}' にデータをロードします")
    total_added = 0
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(documents=batch, embedding=embeddings)
        total_added += len(batch)
        print(f"バッチ {i//batch_size + 1} を追加しました（{len(batch)}件）")
    
    print(f"\n合計 {total_added} 件のドキュメントを追加しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


既存のコレクション 'mycollection_1000' は存在しませんでした
コレクション 'mycollection_1000' を作成しました
現在のコレクション一覧:
- mycollection_500
- mycollection_100
- mycollection_1000
- mycollection_200
コレクション 'mycollection_1000' にデータをロードします
バッチ 1 を追加しました（100件）
バッチ 2 を追加しました（100件）
バッチ 3 を追加しました（100件）
バッチ 4 を追加しました（100件）
バッチ 5 を追加しました（100件）
バッチ 6 を追加しました（98件）

合計 598 件のドキュメントを追加しました


### ベクトル検索のテスト

In [8]:
result = vector_store.similarity_search(query, k=top_k)

In [9]:
for i, doc in enumerate(result[:top_k]):
    print(f"\n=== 検索結果 {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== 検索結果 1 ===
Content: 『メイジアン‧カンパニー』からの登場⼈物で、かなりプライドの⾼い性格で、元⽼院四⼤⽼の⼀⼈である樫和主鷹の側近を務めることで、プライドを満⾜させている。国家機関関係者独⽴魔装⼤隊⾵間 ⽞信（かざま はるのぶ）声 - ⼤川透[21]陸軍⼀〇⼀旅団‧独⽴魔装⼤隊隊⻑で、階級は少佐→中佐→⼤佐。その戦歴と率いる部隊の特殊性から、軍内では階級以上の待遇を受けている。九重⼋雲⾨下の筆頭であり、師と同じく「忍術使い」に分類される古式魔法師だが、最も得意とするのは弟⼦⼊り前から修得していた「天狗術」である。達也の兄弟⼦であると同時に、達也とまともに戦いあえる数少ない⼈物の⼀⼈。魔法師としてはBランクのライセンスを持つ。20代前半のころ、⼤越戦争時に⼤亜連合軍の先遣隊となった⾼麗軍に対し、ゲリラ戦で対抗していたベトナム軍に合流し、「⼤天狗」の異名で呼ばれるようになる。ただし、当時の軍⾸脳部は、⼤亜連合との正⾯衝突を避けたかったため、帰国後は出世コースから外された。旅団⻑の佐伯広海は⼤越戦争以来の気⼼の知れた上司で、内情の壬⽣勇三は兵舎を共にした戦友である。達也とは2092年の沖縄で事件に巻き込まれた司波親⼦を事情聴取した際に知り合い、その後の⼤亜連合の沖縄侵攻時に達也と共闘したことから「質量爆散（マテリアル‧バースト）」を⽬撃することになった。後⽇、国防軍の代表として佐伯と共に四葉家との達也の貸与交渉に同席している。以来、軍では達也の直属の上司となっており達也に⼋雲を紹介したのも⾵間である。達也が四葉家次期当主の婚約者と公表されてからは達也の戦⼒に依存する危険性を感じ、距離を取ることを選択する。しかし、権⼒欲を持ち始めた佐伯の達也に対する利敵⾏為や達也が苦境に陥っている状況にも佐伯の⽴場を尊重して⼿助けを⾏わなかったことで達也への罪悪感と彼との協⼒関係が壊れることへの恐れを抱く。達也が独⽴魔装⼤隊を脱退することを告げた際には佐伯の命令で柳と共に達也を拘束しようとしたが、逆に⾏動不能にされ敗北。師である⼋雲に勝利した⼿腕を称賛した。それ以降、達也に敵愾⼼を抱くようになった上官の命令に、その本⾳を⾒透かしながらも軍⼈として従う。
文字数: 920
Source: data\Wikipedia-魔法科高校の劣等生.pdf

=== 検索結